<a href="https://colab.research.google.com/github/ekvirika/WalmartRecruiting/blob/main/notebooks/model_experiment_sarima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn mlflow

# Set up Kaggle API
!pip install kaggle

In [3]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [5]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.models.signature import infer_signature
import warnings
warnings.filterwarnings('ignore')


In [7]:
!pip install dagshub mlflow --quiet
!pip install optuna
!pip install pmdarima
# !pip install --upgrade numpy scipy statsmodels pmdarima

In [8]:
import mlflow
from dagshub import dagshub_logger
import os

# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "ekvirika"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0adb1004ddd4221395353efea2d8ead625e26197"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

In [9]:
import mlflow
import mlflow.sklearn
import mlflow.pytorch
import dagshub
import os
from typing import Dict, Any, Optional
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import tempfile

In [10]:
!pip install darts
!pip install mlflow
!pip install dagshub

In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class BaseMerger(BaseEstimator, TransformerMixin):

    def __init__(self, features_df, stores_df):
        self.features_df = features_df.copy()
        self.stores_df = stores_df.copy()
        self.features_df["Date"] = pd.to_datetime(self.features_df["Date"])
        self.stores_df["Store"] = self.stores_df["Store"].astype(int)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])
        X["Store"] = X["Store"].astype(int)
        merged = X.merge(self.features_df, on=["Store", "Date"], how="left")
        merged = merged.merge(self.stores_df, on="Store", how="left")
        return merged

class FeatureAdder(BaseEstimator, TransformerMixin):

    def __init__(self, is_train=True):
        self.is_train = is_train

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"], errors="coerce")

        if self.is_train and "Weekly_Sales" in X.columns:
            for lag in [1, 2, 4, 52]:
                X[f"lag_{lag}"] = X.groupby("Store")["Weekly_Sales"].shift(lag)
            X["rolling_mean_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(4).mean()
            X["rolling_std_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(4).std()

        markdown_cols = [col for col in X.columns if "MarkDown" in col]
        X[markdown_cols] = X[markdown_cols].fillna(0)

        X["Year"] = X["Date"].dt.year
        X["Month"] = X["Date"].dt.month
        X["Week"] = X["Date"].dt.isocalendar().week
        X["DayOfWeek"] = X["Date"].dt.dayofweek
        X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
        X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
        X["Quarter"] = X["Date"].dt.quarter

        return X

class MissingValueFiller(BaseEstimator, TransformerMixin):

    def __init__(self, is_train=True):
        self.q_low = None
        self.q_high = None
        self.is_train = is_train

    def fit(self, X, y=None):
        if self.is_train and "Weekly_Sales" in X.columns:
            self.q_low = X["Weekly_Sales"].quantile(0.01)
            self.q_high = X["Weekly_Sales"].quantile(0.99)
        return self

    def transform(self, X):
        X = X.copy()
        X["Date"] = pd.to_datetime(X["Date"])

        if self.is_train and "Weekly_Sales" in X.columns:
            X = X[X["Weekly_Sales"] > 0]
            if self.q_low is not None and self.q_high is not None:
                X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

        return X

class CategoricalEncoder(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.fill_values = {}

    def fit(self, X, y=None):
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if X[col].isnull().any():
                self.fill_values[col] = X[col].mean()
        return self

    def transform(self, X):
        X = X.copy()

        for col, fill_value in self.fill_values.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)

        if "Type" in X.columns:
            X["Type"] = X["Type"].map({"A": 3, "B": 2, "C": 1}).fillna(0)

        if "IsHoliday" in X.columns:
            X["IsHoliday"] = X["IsHoliday"].astype(int)

        X = X.dropna()
        return X


In [13]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('merge', BaseMerger(features, stores)),
    ('value_fill', MissingValueFiller()),
    ('feature_add', FeatureAdder()),
    ('cat_encoder', CategoricalEncoder())
])

processed_df = pipeline.fit_transform(train)

grouped = processed_df.groupby("Store")
train_dict = {store: df for store, df in grouped}

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
import pandas as pd
import numpy as np
from numpy.linalg import LinAlgError
import warnings
from tqdm import tqdm
from statsmodels.tsa.statespace.sarimax import SARIMAX

def train_store_models_sarima(p, d, q, P, D, Q, s, train_weeks):
    mape_array = []
    store_models = {}

    for store in tqdm(train_dict.keys()):
        df = train_dict[store]

        if not isinstance(df, pd.DataFrame):
            df = pd.DataFrame(df)

        try:
            # Prepare time series: align weeks to Monday, aggregate weekly sales
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date'] = df['Date'] - pd.to_timedelta(df['Date'].dt.weekday, unit='D')
            df = df.groupby('Date', as_index=False)['Weekly_Sales'].sum()
            df = df.sort_values('Date')

            # Fill missing weeks (reindex) with zero sales
            full_index = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='W-MON')
            df = df.set_index('Date').reindex(full_index).fillna(0).rename_axis('Date').reset_index()

            # Split train and validation
            train_data = df['Weekly_Sales'].iloc[:train_weeks]
            val_data = df['Weekly_Sales'].iloc[train_weeks:]

            # Fit SARIMA
            model = SARIMAX(train_data,
                            order=(p, d, q),
                            seasonal_order=(P, D, Q, s),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
            model_fit = model.fit(disp=False)

            # Predict validation period
            preds = model_fit.predict(start=train_weeks, end=train_weeks + len(val_data) - 1)

            # Calculate MAPE (ignore zero actuals)
            actual = val_data.values
            predicted = preds.values
            mask = actual != 0
            mape = np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100

            mape_array.append(mape)
            store_models[store] = model_fit

        except (ValueError, LinAlgError, RuntimeError) as e:
            warnings.warn(f"Store {store} failed SARIMA({p},{d},{q})x({P},{D},{Q},{s}): {e}")
            continue

    return mape_array, store_models


In [ ]:
import mlflow
import numpy as np
from datetime import datetime
from tqdm import tqdm

# Your SARIMA param grid — add or change as you like:
param_list = [
    (1, 1, 1, 1, 1, 1, 52),  # weekly data with yearly seasonality
    (2, 1, 2, 1, 1, 1, 52),
    (1, 0, 1, 1, 0, 1, 52),
    # ... add more combinations as needed
]

mlflow.set_experiment("SARIMA_Parameter_Optimization")

best_models = {}
best_params = None
lowest_mape = float('inf')

with mlflow.start_run(run_name=f"SARIMA_Grid_Search_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):

    mlflow.log_param("model_type", "SARIMA")
    mlflow.log_param("validation_weeks", 123)
    mlflow.log_param("param_search_space", str(param_list))
    mlflow.log_param("num_combinations", len(param_list))

    for params in tqdm(param_list):
        p, d, q, P, D, Q, s = params

        with mlflow.start_run(run_name=f"SARIMA({p},{d},{q})x({P},{D},{Q},{s})", nested=True):
            mlflow.log_params({'p': p, 'd': d, 'q': q, 'P': P, 'D': D, 'Q': Q, 's': s})

            mape_array, store_models = train_store_models_sarima(p, d, q, P, D, Q, s, 123)
            mean_mape = np.mean(mape_array)
            mape_std = np.std(mape_array)

            mlflow.log_metric("mean_mape", mean_mape)
            mlflow.log_metric("mape_std", mape_std)
            mlflow.log_metric("successful_stores", len(store_models))

            print(f"Params: {params}, Mean MAPE: {mean_mape:.4f}, Std: {mape_std:.4f}")

            if mean_mape < lowest_mape:
                best_models = store_models
                best_params = params
                lowest_mape = mean_mape
                mlflow.log_metric("is_best_model", 1)
            else:
                mlflow.log_metric("is_best_model", 0)

    # Log best params and save best models
    if best_params:
        mlflow.log_params({'best_p': best_params[0], 'best_d': best_params[1], 'best_q': best_params[2],
                           'best_P': best_params[3], 'best_D': best_params[4], 'best_Q': best_params[5], 'best_s': best_params[6]})
        mlflow.log_metric("best_mean_mape", lowest_mape)

        with open("final_best_sarima_models.pkl", "wb") as f:
            import pickle
            pickle.dump(best_models, f)
        mlflow.log_artifact("final_best_sarima_models.pkl")


 73%|███████▎  | 33/45 [04:34<01:45,  8.83s/it]

In [ ]:
import pandas as pd

predictions = {}
train_end_date = pd.to_datetime(train['Date'].max())

for idx, entry in tqdm(test.iterrows(), total=len(test)):
    store = entry['Store']
    dept = entry['Dept']
    date = pd.to_datetime(entry['Date'])

    pred_weeks = (date - train_end_date).days // 7

    model_fit = best_models.get(store)
    if model_fit is None:
        # handle missing model if needed
        continue

    # Predict up to pred_weeks steps ahead
    preds = model_fit.predict(start=len(model_fit.data.endog),
                              end=len(model_fit.data.endog) + pred_weeks - 1)
    prediction = preds.iloc[-1]

    # If you have sale_proportions for dept adjustments, apply here
    if 'sale_proportions' in globals() and dept in sale_proportions:
        prediction *= sale_proportions[dept]
    else:
        prediction /= len(sale_proportions) if 'sale_proportions' in globals() else 1

    predictions[(store, dept, date)] = prediction


In [ ]:
submission_df = pd.DataFrame([
    {
        'Id': f"{store}_{dept}_{date.strftime('%Y-%m-%d')}",
        'Weekly_Sales': weekly_sales
    }
    for (store, dept, date), weekly_sales in predictions.items()
])

submission_df.to_csv('submission_arima.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission_arima.csv')
